In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import math

In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [3]:
district_old = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [4]:
str_begin = 'http://www.cian.ru/cat.php?deal_type=sale'
str_finish = '&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
str_tmp = '&district%5B{}%5D={}'
str_middle =''
district = []
for i in range(120):    
    str_middle = str_tmp.format(i,13+i)    
    district.append(str_begin + str_middle + str_finish)

In [6]:
links = []
for i in range(120):
    for page in range(1, 2):
        page_url =  district[i].format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                links.append(link)
        print(len(list(set(links))))

28
56
84
112
140
168
196
224
252
280
308
336
363
391
419
447
475
503
531
559
584
612
639
667
695
723
751
779
807
835
863
889
917
945
973
1001
1029
1057
1084
1111
1139
1167
1194
1222
1250
1275
1301
1329
1357
1385
1413
1439
1467
1495
1523
1551
1579
1607
1635
1663
1689
1717
1745
1773
1801
1829
1857
1885
1913
1940
1968
1996
2022
2049
2077
2105
2133
2161
2189
2217
2245
2273
2301
2329
2357
2385
2413
2441
2469
2497
2525
2553
2581
2609
2637
2665
2693
2721
2749
2775
2803
2831
2859
2887
2915
2943
2971
2999
3027
3055
3083
3111
3139
3167
3195
3223
3251
3279
3307
3335


In [8]:
links = list(set(links))
len(links)

3335

In [9]:
def getURL(flat_page):     
    return flat_url

In [10]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

In [11]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lat, lon

In [12]:
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    if (len(room_number) == 0):
        room_number = 6
    else:
        room_number = int(room_number)            
    return room_number

In [13]:
def getWalk(flat_page):
    metrodist = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    metrodist = html_stripper(metrodist)
    walk = 'None'
    for i in re.split('\n', metrodist):
        if 'пешком' in i:
            walk = 1
            break
        elif 'на транспорте' in i:
            walk = 0
            break
        elif 'на машине' in i:
            walk = 0
            break            
    return walk

In [14]:
def getMetrdist(flat_page):
    metrodist = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    metrodist = html_stripper(metrodist)
    distance = ''
    for i in re.split('\n', metrodist):
        if 'мин' in i:
            break
        else:
            distance += i
    distance = "".join(distance.split())
    return distance

In [15]:
def getTotsp(flat_page, table):
    totspField = re.split('Общая площадь:|Площадь комнат', table)[1]
    totsp_list = []
    for item in totspField:
        if item=='м':
            break
        if item.isdigit():
            totsp_list.append(item)
        if (item ==','):
            totsp_list.append('.')
    if len(totsp_list)==0:
        totsp = 'None'
    else:
        totsp = float("".join(totsp_list))
    return totsp

In [16]:
def getLivesp(flat_page, table):
    livespField = re.split('Жилая площадь:|Площадь кухни', table)[1]
    livesp_list = []
    for item in livespField:
        if item=='м':
            break
        if item.isdigit():
            livesp_list.append(item)
        if (item ==','):
            livesp_list.append('.')
    if len(livesp_list)==0:
        livesp = 'None'
    else:
        livesp = float("".join(livesp_list))
    return livesp

In [17]:
def getKitsp(flat_page, table):
    kitspField = re.split('Площадь кухни:|Санузел:|Раздельных санузлов', table)[1]
    kitsp_list = []
    for item in kitspField:
        if item=='м':
            break
        if (item.isdigit()):
            kitsp_list.append(item)
        if (item ==','):
            kitsp_list.append('.')
    if len(kitsp_list)==0:
        kitsp = 'None'
    else:
        kitsp = float("".join(kitsp_list))
    return kitsp

In [18]:
def getFloors(flat_page, table):    
    floorsField = re.split('Этаж:|Тип дома', table)[1]
    floors_list = []
    for item in floorsField:
        if item=='/':
            floors_list.append(',')
        if item.isdigit():
            floors_list.append(item)
    floorcount = "".join(floors_list)  
    floorcount = [x.strip() for x in floorcount.split(',')]
    if (len(floorcount)==1):
        floor = int(floorcount[0])
        nfloors = 'None'
    else:
        floor = int(floorcount[0])
        nfloors = int(floorcount[1]) 
    return floor, nfloors

In [19]:
def getTelephone(flat_page, table):   
    telephoneField = re.split('Телефон:|Вид из окна', table)[1]
    telephone_list = []
    for item in telephoneField:
        if item.isalpha():
            telephone_list.append(item)
    telephone = "".join(telephone_list)
    if len(telephone_list)==0:
        tel = 'None'
    elif (telephone =='да'):
        tel = 1
    elif (telephone =='нет'):
        tel = 0
    else:
        tel = 'None'
    return tel

In [20]:
def getBalcony(flat_page, table):   
    balconyField = re.split('Балкон:|Лифт', table)[1]
    balcony_list = []
    for item in balconyField:
        if item.isdigit():
            balcony_list.append(item)
    balcony = "".join(balcony_list)
    if len(balcony_list)==0:
        bal = 0
    else:
        bal = 1
    return bal

In [21]:
def getNewBrick(flat_page, table):   
    newbrickField = re.split('Тип дома:|Высота потолков:|Тип продажи:', table)[1]
    newbrick_list = []
    for item in newbrickField:
        if item.isalpha():
            newbrick_list.append(item)
        if (item ==','):
            newbrick_list.append(item)
    newbrick = "".join(newbrick_list)  
    newbrick = [x.strip() for x in newbrick.split(',')]
    new = None
    if (newbrick[0]=='вторичка'):
        new = 0
    elif (newbrick[0]=='новостройка'):
        new = 1
    if (len(newbrick)==1):
        brick = 'None'
    elif ((newbrick[1]=='кирпичный') | (newbrick[1]=='монолитный') | (newbrick[1]=='монолитнокирпичный') | (newbrick[1]=='кирпичномонолитный')):
        brick = 1
    else: brick = 0
    return new, brick

In [22]:
def getDist(flat_page, table): 
    phi1 = 55.751244
    phi2 = getCoords(flat_page)[0]
    lambda1 = 37.618423
    lambda2 = getCoords(flat_page)[1]
    d_phi = phi1 - phi2
    d_lambda = lambda1 - lambda2
    dist = 6371*math.radians(np.arctan((((np.cos(phi2)*np.sin(d_lambda))**2 + (np.cos(phi1)*np.sin(phi2)-np.sin(phi1)*np.cos(phi2)*np.cos(d_lambda))**2)**0.5)/
    (np.sin(phi1)*np.sin(phi2) + np.cos(phi1)*np.cos(phi2)*np.cos(d_lambda))))
    return ("%.3f" % dist)

In [23]:
CIAN = pd.DataFrame(columns=['URL', 'Rooms', 'Price', 'Totsp', 'Livesp', 'Kitsp', 'Dist', 'Metrdist', 'Walk', 'Brick', 'Tel', 'Bal',
                            'Floor', 'Nfloors', 'New'])

for page in range(len(links)):
    print(page)
    flat_url = 'http://www.cian.ru/sale/flat/' + str(links[page]) + '/'
    flat_page = requests.get(flat_url)
    flat_page = flat_page.content
    flat_page = BeautifulSoup(flat_page, 'lxml')
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    CIAN = CIAN.append({'Price':getPrice(flat_page), 'Rooms':getRoom(flat_page), 'Walk':getWalk(flat_page),
                        'Metrdist':getMetrdist(flat_page), 'Totsp':getTotsp(flat_page, table), 
                       'Livesp':getLivesp(flat_page, table), 'Kitsp':getKitsp(flat_page, table),
                       'Tel': getTelephone(flat_page, table), 'Bal':getBalcony(flat_page, table), 
                       'Dist':getDist(flat_page, table), 'URL': getURL(flat_page),
                       'Floor': getFloors(flat_page, table)[0], 'Nfloors': getFloors(flat_page, table)[1],
                       'New': getNewBrick(flat_page, table)[0], 'Brick': getNewBrick(flat_page, table)[1]}, ignore_index=True)    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
CIAN

,URL,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New
0,http://www.cian.ru/sale/flat/150955746/,1.0,8900000.0,39.0,19.1,8,11.829,4,1,0,1,1.0,13.0,17,0.0
1,http://www.cian.ru/sale/flat/150197725/,1.0,4278000.0,27.6,14,5.6,12.643,7,0,1,None,0.0,15.0,19,1.0
2,http://www.cian.ru/sale/flat/148939063/,1.0,4013900.0,35.3,15,12,24.687,15,0,0,None,0.0,3.0,17,1.0
3,http://www.cian.ru/sale/flat/150460380/,4.0,29800000.0,228.0,125,21.8,13.669,None,None,None,None,1.0,15.0,16,0.0
4,http://www.cian.ru/sale/flat/149550321/,2.0,5800000.0,42.0,27,6,9.443,10,0,0,1,1.0,2.0,5,0.0
5,http://www.cian.ru/sale/flat/149846199/,2.0,8990000.0,46.0,28,6,7.818,11,1,1,0,1.0,5.0,5,0.0
6,http://www.cian.ru/sale/flat/36303905/,1.0,5090000.0,32.0,17.6,6,14.816,26,0,0,1,1.0,5.0,5,0.0
7,http://www.cian.ru/sale/flat/149696544/,2.0,13950000.0,72.5,37.8,11,24.646,None,None,1,1,1.0,3.0,5,0.0
8,http://www.cian.ru/sale/flat/150206839/,1.0,8000000.0,37.0,20,9,5.338,8,0,1,1,1.0,2.0,14,0.0
9,http://www.cian.ru/sale/flat/148879179/,2.0,6650000.0,40.0,24,6,15.019,5,1,0,None,1.0,10.0,12,0.0


In [25]:
CIAN[['Rooms','Price','Bal','Floor','New']] = CIAN[['Rooms','Price','Bal','Floor','New']].astype(int)
CIAN

,URL,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New
0,http://www.cian.ru/sale/flat/150955746/,1,8900000,39.0,19.1,8,11.829,4,1,0,1,1,13,17,0
1,http://www.cian.ru/sale/flat/150197725/,1,4278000,27.6,14,5.6,12.643,7,0,1,None,0,15,19,1
2,http://www.cian.ru/sale/flat/148939063/,1,4013900,35.3,15,12,24.687,15,0,0,None,0,3,17,1
3,http://www.cian.ru/sale/flat/150460380/,4,29800000,228.0,125,21.8,13.669,None,None,None,None,1,15,16,0
4,http://www.cian.ru/sale/flat/149550321/,2,5800000,42.0,27,6,9.443,10,0,0,1,1,2,5,0
5,http://www.cian.ru/sale/flat/149846199/,2,8990000,46.0,28,6,7.818,11,1,1,0,1,5,5,0
6,http://www.cian.ru/sale/flat/36303905/,1,5090000,32.0,17.6,6,14.816,26,0,0,1,1,5,5,0
7,http://www.cian.ru/sale/flat/149696544/,2,13950000,72.5,37.8,11,24.646,None,None,1,1,1,3,5,0
8,http://www.cian.ru/sale/flat/150206839/,1,8000000,37.0,20,9,5.338,8,0,1,1,1,2,14,0
9,http://www.cian.ru/sale/flat/148879179/,2,6650000,40.0,24,6,15.019,5,1,0,None,1,10,12,0


In [26]:
CIAN.to_csv('CIAN_Moscow.csv', sep=',', header=True, index=False)